# Laboratorio 6: Prediccion de infeccion por malware
### creación de modelo DGA - codigo obtenido de la clase

In [4]:
import pandas as pd
import numpy as np
import caracteristicasDerivadas
from caracteristicasDerivadas import H_entropy, proporcionVocalesConsonantes, posicionPrimerDigito
#import  matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling as pp
from pandas_profiling import ProfileReport
import sklearn
from sklearn import metrics, model_selection, tree

In [5]:
import sklearn
from sklearn import metrics, model_selection, tree

In [6]:
#Carga de datos
df = pd.read_csv('dga_data_small.csv')

In [7]:
df.head(5)

,isDGA,domain,host,subclass
0,dga,tyopcrkqgxcfm,tyopcrkqgxcfm.co.uk,cryptolocker
1,dga,72j5rn1l9mzleo6203v1ogenfl,72j5rn1l9mzleo6203v1ogenfl.org,gameoverdga
2,dga,thenrest,thenrest.net,nivdort
3,dga,15ihbm71utcnfa8dk1mmgoobl9,15ihbm71utcnfa8dk1mmgoobl9.org,gameoverdga
4,dga,x1d6ou7e7kofk60ayhq74x7e,x1d6ou7e7kofk60ayhq74x7e.net,gameoverdga


In [8]:
#Preprocesamiento

#Eliminacion de caracteristicas irrelevante o repetidas
df.drop(['host','subclass'], axis=1, inplace=True)

#Codificacion de variable objetivo
df['isDGA'] = df['isDGA'].replace(to_replace='dga', value=1)
df['isDGA'] = df['isDGA'].replace(to_replace='legit', value=0)



In [9]:
#Derivar caracteristicas
# 1. Longitud
# 2. Cantidad de digitos
# 3. Calculo de entropia (Shannon), es decir la cantidad de "informacion" que se puede obtener de una variable
# 4. Posicion del primer digito en la cadena
# 5. Proporcion de vocales - consonantes

df['longitud'] = df['domain'].str.len()
df['digitos'] = df['domain'].str.count('[0-9]')
df['entropia'] = df['domain'].apply(H_entropy)
df['proporcionVocalesConsonantes'] = df['domain'].apply(proporcionVocalesConsonantes)
df['posicionPrimerDigito'] = df['domain'].apply(posicionPrimerDigito)

df.drop(['domain'], axis=1, inplace=True)


In [10]:
df.head()

,isDGA,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito
0,1,13,0,3.546594,0.083333,0
1,1,26,10,4.132944,0.333333,1
2,1,8,0,2.500000,0.333333,0
3,1,26,7,4.180833,0.357143,1
4,1,24,9,3.834963,0.666667,2


In [11]:
df_final = df

In [12]:
target = df_final['isDGA']
feature_matrix = df_final.drop(['isDGA'], axis=1)

In [13]:
print('Final features:', feature_matrix.columns)

Final features: Index(['longitud', 'digitos', 'entropia', 'proporcionVocalesConsonantes',
       'posicionPrimerDigito'],
      dtype='object')


In [14]:
feature_matrix.head()

,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito
0,13,0,3.546594,0.083333,0
1,26,10,4.132944,0.333333,1
2,8,0,2.500000,0.333333,0
3,26,7,4.180833,0.357143,1
4,24,9,3.834963,0.666667,2


In [15]:
feature_matrix_train, feature_matrix_test, target_train, target_test = model_selection.train_test_split(feature_matrix, target, test_size=0.25, random_state=31)

In [16]:
clf = tree.DecisionTreeClassifier()


In [17]:
clf = clf.fit(feature_matrix_train, target_train)

In [18]:
feature_matrix_train.count()

longitud                        1500
digitos                         1500
entropia                        1500
proporcionVocalesConsonantes    1500
posicionPrimerDigito            1500
dtype: int64

In [19]:
feature_matrix_test.count()

longitud                        500
digitos                         500
entropia                        500
proporcionVocalesConsonantes    500
posicionPrimerDigito            500
dtype: int64

In [20]:
target_pred = clf.predict(feature_matrix_test)

In [21]:
print(metrics.accuracy_score(target_test, target_pred))


0.846


In [22]:
print('Matriz de confusion /n',metrics.confusion_matrix(target_test, target_pred))


Matriz de confusion /n [[235  23]
 [ 54 188]]


In [23]:
print(metrics.classification_report(target_test, target_pred, target_names=['legit', 'dga']))


              precision    recall  f1-score   support

       legit       0.81      0.91      0.86       258
         dga       0.89      0.78      0.83       242

    accuracy                           0.85       500
   macro avg       0.85      0.84      0.84       500
weighted avg       0.85      0.85      0.85       500



In [29]:
#from sklearn.external import joblib
import six
import sys
sys.modules['sklearn.externals.six'] = six
import joblib
sys.modules['sklearn.externals.joblib'] = joblib


In [30]:
joblib.dump(clf, 'Modelo_DGA_entrenado.pkl')

['Modelo_DGA_entrenado.pkl']